# Robolog Reader Basics

This tutorial covers the basics of the robolog reader. But first, let's make sure we are in the repo root directory.

In [1]:
import os
from pathlib import Path

current_dir = Path.cwd()
project_root = None

while current_dir != current_dir.parent:
    if (current_dir / '.git').exists():
        project_root = current_dir
        break
    current_dir = current_dir.parent

os.chdir(project_root)
print(f"cd {project_root!s}")

cd /home/shouheng/Development/repos/bagel


This tutorial reads a ROS2 bag, which requires an active ROS2 environment.

**Important**: Before launching JupyterLab, you must first source your ROS2 installation in the terminal. Using ROS2 Iron for example:

```sh
source /opt/ros/iron/setup.sh
```

If JupyterLab is already running, you need to close it, run the `source` command, and restart JupyterLab from the same terminal.

### Creating a Reader

To create a reader, pass the robolog path to the `make_topic_message_reader` factory method. The method automatically identifies the log format (e.g., a ROS2 bag) and returns the correct reader, such as a `TopicMessageReader`.

In [2]:
from src.reader import factory

reader = factory.make_topic_message_reader("doc/tutorials/data/ros2")

### Reading Metadata

All robolog readers share common metadata properties across different log formats (e.g., ROS2, ROS1, PX4 ULog).

For example, `reader.start_seconds` provides the start time for any supported robolog.

In [3]:
print(f"Robolog ID: '{reader.robolog_id}'")
print(f"Path: '{reader.path}'")
print(f"Duration in seconds: {reader.duration_seconds}")
print(f"Number of messages: {reader.total_message_count}")
print(f"Start time: {reader.start_seconds}")
print(f"End time: {reader.end_seconds}")

Robolog ID: '281e260f-0fc9-57e6-8c33-787dc436009f'
Path: '/home/shouheng/Development/repos/bagel/doc/tutorials/data/ros2'
Duration in seconds: 2.9702672958374023
Number of messages: 15
Start time: 1689969664.041739
End time: 1689969667.0120063


You can view the topics contained in this robolog.

In [4]:
reader.topics

['/events/write_split', '/fluid_pressure', '/rosout', '/parameter_events']

You can also view the message type of each topic.

In [5]:
reader.type_names

{'/fluid_pressure': 'sensor_msgs/msg/FluidPressure',
 '/parameter_events': 'rcl_interfaces/msg/ParameterEvent',
 '/events/write_split': 'rosbag2_interfaces/msg/WriteSplitEvent',
 '/rosout': 'rcl_interfaces/msg/Log'}

... and the number of messages of each topic.

In [6]:
reader.message_counts

{'/fluid_pressure': 7,
 '/parameter_events': 1,
 '/events/write_split': 0,
 '/rosout': 7}

### Reading Logging Messages

The reader provides a convenient property to access the robolog's logging messages. In this example, we visualize them as a pandas DataFrame.

In [7]:
import pandas as pd

pd.DataFrame([msg.to_dict() for msg in reader.logging_messages])

,robolog_id,timestamp_seconds,level,message,numeric_level,topic,name,file,function,line
0,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,INFO,Press SPACE for pausing/resuming,20,/rosout,rosbag2_recorder,./src/rosbag2_transport/recorder.cpp,RecorderImpl,171
1,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,INFO,Event publisher thread: Starting,20,/rosout,rosbag2_recorder,./src/rosbag2_transport/recorder.cpp,event_publisher_thread_main,304
2,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,INFO,Listening for topics...,20,/rosout,rosbag2_recorder,./src/rosbag2_transport/recorder.cpp,record,293
3,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,INFO,Subscribed to topic '/rosout',20,/rosout,rosbag2_recorder,./src/rosbag2_transport/recorder.cpp,subscribe_topic,433
4,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,INFO,Subscribed to topic '/events/write_split',20,/rosout,rosbag2_recorder,./src/rosbag2_transport/recorder.cpp,subscribe_topic,433
5,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,INFO,Subscribed to topic '/parameter_events',20,/rosout,rosbag2_recorder,./src/rosbag2_transport/recorder.cpp,subscribe_topic,433
6,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,INFO,Subscribed to topic '/fluid_pressure',20,/rosout,rosbag2_recorder,./src/rosbag2_transport/recorder.cpp,subscribe_topic,433


### Reading All Metadata

You can also read all metadata in this robolog as a JSON-serializable dictionary.

In [8]:
reader.metadata

{'bag_size': 20642,
 'compression_format': '',
 'compression_mode': '',
 'message_count': 15,
 'relative_file_paths': ['fluid_message_0.mcap'],
 'storage_identifier': 'mcap',
 'version': 7,
 'files': [{'message_count': 15,
   'path': 'fluid_message_0.mcap',
   'starting_time_seconds': 1689969664.041739,
   'duration_seconds': 2.970267}],
 'duration_seconds': 2.97026721,
 'starting_time_seconds': 1689969664.041739,
 'topics_with_message_count': [{'message_count': 7,
   'topic_metadata': {'name': '/fluid_pressure',
    'offered_qos_profiles': '- history: 3\n  depth: 0\n  reliability: 1\n  durability: 2\n  deadline:\n    sec: 9223372036\n    nsec: 854775807\n  lifespan:\n    sec: 9223372036\n    nsec: 854775807\n  liveliness: 1\n  liveliness_lease_duration:\n    sec: 9223372036\n    nsec: 854775807\n  avoid_ros_namespace_conventions: false',
    'serialization_format': 'cdr',
    'type': 'sensor_msgs/msg/FluidPressure',
    'type_description_hash': 'RIHS01_22dfb2b145a0bd5a31a1ac3882a1b321